多くの場合、データは複数のファイルやデータベースに分かれて保存されていたり、簡単には分析できない形式になっていたりする  
この章では、データの結合や連結、変形などといった操作に役立つツールに焦点を当てていく

# 8.1  階層型インデックス

階層型インデックスとは、複数のインデックスの階層を軸に持たせることができる機能で、pandasの重要な機能のひとつになる  
やや抽象的な言い方をすると、階層型インデックスは、高次元のデータをより低次元の形で扱う方法を提供する

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                  index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                         [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -1.507787
   2    1.045325
   3   -2.135566
b  1   -1.505431
   3   -2.383749
c  1   -1.087562
   2   -1.209830
d  2    0.495330
   3   -0.436243
dtype: float64

簡単な例として、まずはリストのリストをインデックスとして指定してシリーズを作る  

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

階層型インデックスを持つシリーズオブジェクトのインデックスオブジェクトを返すと、MultiIndexオブジェクトとして、行ごとのインデックスがペアになったタプルを返す

In [4]:
data["b"]

1   -1.505431
3   -2.383749
dtype: float64

In [5]:
data["b": "c"]

b  1   -1.505431
   3   -2.383749
c  1   -1.087562
   2   -1.209830
dtype: float64

In [6]:
data.loc[["b", "d"]]

b  1   -1.505431
   3   -2.383749
d  2    0.495330
   3   -0.436243
dtype: float64

階層型インデックスを持つオブジェクトを使うと、データ部分集合を簡潔に抽出できる  
上のような参照を、俗に部分インデックス参照と呼ぶこともある

In [7]:
data.loc[:, 2]

a    1.045325
c   -1.209830
d    0.495330
dtype: float64

内側の階層を指定して抽出することもできる

In [8]:
data.unstack()

,1,2,3
a,-1.507787,1.045325,-2.135566
b,-1.505431,NaN,-2.383749
c,-1.087562,-1.209830,NaN
d,NaN,0.495330,-0.436243


階層型インデックスは、データの変形や、ピボットテーブル作成のようなグループスペースの操作を行うときに重要な役割を果たす  
例えば、上のようにunstackメソッドを使うことで、先ほどのデータをデータフレームに変形することができる

In [9]:
data.unstack().stack()

a  1   -1.507787
   2    1.045325
   3   -2.135566
b  1   -1.505431
   3   -2.383749
c  1   -1.087562
   2   -1.209830
d  2    0.495330
   3   -0.436243
dtype: float64

逆に、データフレームを階層型インデックスを持つシリーズにするにはstackメソッドを用いることでできる 

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

シリーズだと行のインデックスにしか階層型インデックスを持たせることはできないが、データフレームだと列と行のどちらでも階層型インデックスを持たせることができる

In [11]:
frame.index.names = ["key1", "key2"]

In [12]:
frame.columns.names = ["state", "color"]

In [13]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

階層型インデックスのそれぞれの階層インデックスに名前を付けることができる  
それぞれindexオブジェクト、もしくはcolumnsオブジェクトのnames属性からアクセスしてリストで代入することで名前を付けることができる  
名前をつけると、コンソールの出力にインデックスの名前が表示されるようになる

In [14]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

列の部分インデックス参照に先程付けた名前を使って選択することもできる

In [15]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "colorado"], ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('colorado', 'Green')],
           names=['state', 'color'])

階層型インデックスのデータを作るときに、引数で直接渡すほかにMultiIndexオブジェクトを渡して作成することもでき、この場合再利用をして使いまわすことができる  
MultiIndexオブジェクトはpandasのMultiIndex.from_arraysに階層型インデックスに使いたいリストのリストを渡すことでできる  
またキーワード引数のnamesに階層の名前を付けることもできる

## 8.1.1  階層の順序変更やソート

特定の軸のインデックス階層の順序を変更したり、特定の階層の値によってデータをソートしたりする操作は時々必要になる

In [16]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

特定の階層を入れ替えたいときは、swaplevelメソッドを使って、入れ替えたい2つの階層を渡すことでできる  
階層の指定は階層の名前でも、階層のインデックスでもよい(外側の階層が0、その内側が1)

In [17]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

特定の1つの階層の値を用いてデータをソートしたい場合はsort_indexメソッド使って、キーワード引数のlevelに階層を指定することでできる  
swaplevelメソッドを用いて階層を入れ替えた後に、sort_indexメソッドを使って外側の階層をソートする作業はよく使われる

## 8.1.2  階層ごとの要約統計量

データフレームやシリーズの多くの要約統計量(記述統計量)には、levelオプションを与えることができる  
levelオプションを使うと、その軸での集計対象としたい階層を指定できる

In [19]:
frame.sum(level="key2")

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.sum(level="color", axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

この処理の内部では、pandasのgroupby機構が活用されている

## 8.1.3  データフレームの列をインデックスに使う

データフレームの特定の列の値を行インデックスとして使いたい場合や、その逆のことをしたい場合もある

In [21]:
frame =pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                     "c": ["one", "one", "one", "two", "two", "two", "two"],
                     "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [22]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

データフレームのset_indexメソッドは、指定した1つ以上の列をインデックスとして持った新しいデータフレームを生成する

In [23]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

デフォルトでは、インデックスに使用された列はデータフレームから削除されるが、キーワード引数のdropにFalseを渡すことでインデックスとして使用した列を残すことができる

In [24]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


一方データフレームのreset_indexメソッドはset_indexメソッドとは逆の動作をし、階層型インデックスのそれぞれの階層が列として変換される

# 8.2  データセットの結合とマージ

pandasに含まれるデータは、いろんな方法で結合することができる  
- pandas.mergeという関数は、複数のデータフレームの行同士を1つ以上のキーに基づいて連結する  
このような操作は、SQLなどのリレーショナルデータベースのユーザにとってはなじみ深い操作になる  
SQLでは、データベースのjoinという操作として実装されているのがこれにあたる  
- pandas.concatという関数は、特定の軸に沿ってデータフレームをつなげたり「積み上げ」たりする(つまり縦や横に連結できる)  
- combine_firstというインスタンスメソッドを用いると、重複するデータをもつ複数のオブジェクトをつなぎ合わせて、オブジェクトの欠損値を別のオブジェクトの値で穴埋めできる

## 8.2.1  データフレームをデータベース風に結合する

マージや結合と呼ばれる操作は、複数のデータセットに含まれる行同士を、1つ以上のキーを使ってリンクさせることで、複数のデータセット結びつける操作で、リレーショナルデータベースの中核をなす重要な操作になる  

In [25]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": range(7)})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": range(3)})

In [26]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [27]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


上の2つのデータを使うとする

In [28]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


pandasのmerge関数を使って2つ以上のデータフレームを渡すことで、特定の列の共通した要素を基準にして(上だとkey列)結合したデータフレームを返す  
上のようにdf1にはaやbという要素が複数あるのに対して、df2には1ずつしかない  
このような多対一の結合では上のような結果になり、ひとつしかなかったdf2のaやbのような行が多の数に合わせるように増やされる

In [29]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


上の結果は先程とは変わらないが、明示的にどの列をキー(基準)として扱うかを指示している  
キーを指定しない場合は、merge関数が双方に共通した列名を探しだし自動的にキーとして扱う  
しかし、予想した結果を得やすくするためにもキーを指定したほうがよい

In [30]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": range(7)})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": range(3)})

In [31]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


キーとしたい列名が結合するデータフレーム同士で異なっていた場合、キーとする列を個別で指定することもできる  
merge関数のキーワード引数left_onとright_onにそれぞれのデータフレームの列名を渡すことでできる

これまでのmerge関数を使った結合では、片方にのみ存在した"c"列や"d"列が結合後には無くなっている  
merge関数がデフォルトで行うのは内部結合(inner-join)で、両社のテーブルに共通して存在する要素のみを結果に含めるような結合方法になる  

In [32]:
pd.merge(df1, df2, on="key", how="outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


内部結合以外の方法を使って結合したい場合は、merge関数のキーワード引数howに結合方法を表す文字列を渡すことでできる  
上の場合はhowにouterを渡して外部結合を行い、両方のテーブルの要素を和集合で取り、片方しかない要素に対応するもう一つのテーブルの欄には欠損値が入れられる方法をとっている  
innerやouterの他にleft(左外部結合)とright(右外部結合)があり、leftは左側にあるテーブルの要素をすべて使い、右側にしかない要素は無視される  
rightはleftとは逆の関係になる  
howオプションで指定できる結合方法  

|値|挙動|
|:-|:-|
|inner|2つのテーブルの両方に含まれるキーのみを用いて結合を行う|
|left|左側のテーブルに含まれるキーをすべて用いて結合を行う|
|right|右側のテーブルに含まれるキーをすべて用いて結合を行う|
|outer|2つのテーブルの一方にでも含まれるキーをすべて用いて結合を行う|

In [33]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": range(6)})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": range(5)})

In [34]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [35]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [36]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


merge関数を使って多対多のデータフレーム同士を結合した場合は上のように行の直積の形で結合される  
つまり左のテーブルの要素と右のテーブルの要素の全ての組み合わせを持ったデータフレームが作られる  
上の例で言うと、"b"は左のテーブルに3個、右のテーブルに2個あるので、すべてで6個の組み合わせができる

In [37]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": [1, 2, 3]})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"], 
                      "key2": ["one", "one", "one", "two"],
                      "rval": [4, 5, 6, 7]})

In [38]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [39]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [40]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


複数の列をキーとして組み合わせたい場合は、キーワード引数のonに列名のリストを渡すことでできる

In [41]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


merge関数を用いて結合したときに重複した列名を持ったデータフレーム同士だった場合、自動的に接尾語を付けて名前を変えられてしまう(上の場合key2_xとkey2_y)  
手動でこの後に列名を変えることもできるが、merge関数を呼ぶ際に前もって接尾語を決めることができる  

In [42]:
pd.merge(left, right, on="key1", suffixes=("_left", "_rigth"))

,key1,key2_left,lval,key2_rigth,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


merge関数のキーワード引数のsuffixesに接尾語として付けたい左のテーブルの列名と右のテーブルの列名を渡すことで、重複した列名があった場合に指定した接尾語のついた列名にして返してくれる

merge関数の引数

|引数|説明|
|:-|:-|
|left|マージ対象となる左側のデータフレーム|
|right|マージ対象となる右側のデータフレーム|
|how|"inner"、"outer"、"left"、"right"のいずれかを指定。デフォルトは"inner"|
|on|結合に使う列名。結合対象のデータフレームオブジェクトの両方に存在する列名でなければならない。この引数が指定されず、left_onなどほかの引数でも結合キーが指定されなかった場合は、左右で共通する列名を結合キーとする|
|left_on, right_on|左と右でそれぞれのデータフレームで結合キーとして用いる列名|
|left_index, right_index|左、もしくは右のデータフレームについては、行のインデックスを結合キーとして用いる|
|sort|マージ後のデータを結合キーで辞書順にソートする。デフォルトではTrue。|
|suffixes|重複する列名がある場合に、列名の末尾に付加する文字列のタプル。デフォルトでは(\_x, \_y)|
|copy|Falseの場合は、いくつかの例外を除き、マージにより新たなデータ構造を作るときにデータのコピーを行わないようにする。デフォルトでは常にコピーを行う|
|indicator|マージ後のデータフレームに_mergeという特殊な列を追加する。この列には、各行に含まれる結合されたデータがどちらのデータに由来するかの情報が入れられる。値は"left_only"(右のみ)、"right_only"(左のみ)、"both"(両方)のいずれかになる|

## 8.2.2  インデックスによるマージ

In [43]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": range(6)})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [44]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [45]:
right1

,group_val
a,3.5
b,7.0


In [46]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


マージ対象のキーがデータフレームの列ではなくインデックスとして使われているものをキーとしたい場合は、キーワード引数のleft_indexやright_indexにTrueを渡すことでインデックスをマージのキーとして使うことができる

In [47]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"], 
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"], 
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=["event1", "event2"])

In [48]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [49]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

階層型インデックスを持ったデータの場合、インデックスによる結合は必然的に複数のキーを用いたマージとなるため、少し複雑になる  
上のデータを使って進める

In [50]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [51]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


階層型インデックスのデータを使ったマージは上のようになり、片方が階層型インデックスを持っていた場合、もう片方のデータのマージに用いるキーは複数の列をリストとして渡さなければならない

In [52]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"])

In [53]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [54]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [55]:
pd.merge(left2, right2, left_index=True, right_index=True, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


2つのデータフレームの結合で、両方のインデックスをキーとしてマージすることもできる

In [56]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


インデックスによるマージを簡単に行える、データフレームのjoinメソッドというものがある  
joinメソッドに結合したいデータフレームを渡すことででき、同一のインデックスや類似したインデックスを持ったたくさんのデータフレームオブジェクトを結合できる  
ただし、それらの結合するデータフレームオブジェクトには重複した列名が使われていると使うことができない

In [57]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [58]:
right1

,group_val
a,3.5
b,7.0


In [59]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


データフレームのjoinメソッドが行うのは、結合キーを用いた左結合(left join)になる  
つまり、呼び出し元のデータフレームオブジェクトの行のインデックスが完全に保存される  
joinメソッドでは、onに呼び出し元のデータフレームの列名を渡すことで、引数に渡されたデータフレームのインデックスを、呼び出し元のデータフレームの列に対して結合することもできる

In [60]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [61]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [62]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [63]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


インデックス同士のマージで、データフレームのjoinメソッドに複数のデータフレームオブジェクトをリストにして渡すと複数のデータフレームを同時にマージすることができる(concat関数を用いた時と同じ効果をもたらす)

## 8.2.3  軸に沿った連結

もう1つのデータ結合操作は、連結、バインド、あるいは積み重ねと呼ばれるものになる

In [64]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [65]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

NumPyにはNumPy配列を連結するconcatenate関数がある  
上のように、配列をリストにして渡すことで配列同士を連結することができる

pandasオブジェクトという観点で見ると、どれにもラベルが付いた軸があるため、配列の連結操作をさらに一般化することができる  
そのため、特に次のような事項を考慮する必要がある  
- 連結に使用しない軸のインデックスがオブジェクト間で異なる場合に、軸間でのインデックスの差異を無視して要素を結合するか、それともインデックスが同じ要素のみを結合するか(すなわち内部結合か外部結合か)
- 連結によって得られたオブジェクトにおいて、連結前の各要素が識別できる必要はあるか
- 「連結対象の軸」に含まれている情報は残しておく必要があるか。多くの場合、データフレームにデフォルトで設定されていた数値ラベルは、連結操作を通じて捨て去ってしまう(新たに採番し直す)のがよい  

pandasのconcat関数を用いると、これらの懸念点にそれぞれに一貫した方法で対処できる

In [66]:
s1 = pd.Series([0, 1], index=["a", "b"])
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"])
s3 = pd.Series([5, 6], index=["f", "g"])

In [67]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

上のようなインデックスに重複のないシリーズをpandasのconcat関数にリストとして渡すと、シリーズ内の値やインデックスが連結される

In [68]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


pandasのconcat関数は、デフォルトではaxis=0の方向に連結されシリーズのまま返す  
axis=1を渡して行方向に連結することもでき、その場合はデータフレームとして返される

In [69]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [70]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [71]:
pd.concat([s1, s4], axis=1, join="inner")

,0,1
a,0,0
b,1,1


連結に用いない方の軸において、ラベルの重複がなかった場合は、もともとの軸のインデックスの集合(外部結合)をソートしたものになる  
代わりにconcat関数のキーワード引数のjoinにinnerを指定することで、共通したインデックスのみの集合(内部結合)を得ることができる  
内部結合を行ったときに片方しかなかったラベル(上の場合fとg行)が結合結果から消える

テキストにあるキーワード引数のjoin_axesは、連結する軸のインデックスを指定することができるが、pandasのバージョン1.00で廃止されたために使えない  
使う場合はpandasのバージョンを下げる必要がある

このような連結操作は、連結によって得られたオブジェクトにおいて、どの要素が連結前の要素と対応するのか識別できない  
この問題を解決するには、連結対象の軸に連結した単純なインデックスではなく、階層型インデックスを設定するとよい

In [72]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

階層型インデックスを作成して設定するには、concat関数のキーワード引数のkeysに上の階層として使うインデックス名を渡すことでできる  
これにより、連結元のデータごとに分けてインデックスを付けることができる

In [73]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


unstackメソッドを使って各行の下の階層のインデックスを各列のインデックスとして変換することもできる(データフレームになる)

In [74]:
pd.concat([s1, s2, s3], axis=1, keys=["one", "two", "three"])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


concat関数でaxis=1を設定して列方向に連結した場合は、keysに与えたインデックス名は行のヘッダ(列のインデックス)として使われる

In [75]:
df1 = pd.DataFrame(np.arange(6).reshape((3, 2)), 
                   index=["a", "b", "c"],
                   columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape((2, 2)),
                   index=["a", "c"],
                   columns=["three", "four"])

In [76]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [77]:
df2

,three,four
a,5,6
c,7,8


In [78]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

データフレームオブジェクトの場合も同様にできる 

In [79]:
pd.concat({"level1": df1, "level2": df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

concat関数にリストではなく、ディクショナリとしてキーにインデックス名、値に連結するデータフレームを渡すと、キーワード引数のkeysにディクショナリのキーを渡すのと同じ効果が得られ階層型インデックスを作成できる

In [80]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"], names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

concat関数にはキーワード引数のnamesを渡すことができ、namesにリストを指定することで、階層ごとに名前を付けることができる

In [81]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=["b", "d", "a"])

In [82]:
df1

,a,b,c,d
0,0.093036,0.582675,-0.922212,0.176834
1,-0.208426,-0.948847,-0.053033,-0.760699
2,1.141578,1.663517,-1.575925,0.116203


In [83]:
df2

,b,d,a
0,-0.089472,-0.008632,-0.705118
1,-1.080989,0.339001,-0.601499


In [84]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.093036,0.582675,-0.922212,0.176834
1,-0.208426,-0.948847,-0.053033,-0.760699
2,1.141578,1.663517,-1.575925,0.116203
3,-0.705118,-0.089472,NaN,-0.008632
4,-0.601499,-1.080989,NaN,0.339001


データフレームの行インデックスにとくに重要な情報が含まれていない場合は、キーワード引数のignore_indexにTrueを渡すことで元のデータのインデックスは無視され、新たにインデックスが0からふり直される

concat関数の引数

|引数|説明|
|:-|:-|
|objs|連結対象とするpandasオブジェクトのリストまたはディクショナリ。唯一の必須の引数で、第一引数として指定なしで渡すことができる|
|axis|連結する方向を表す軸。デフォルトは0(行方向)|
|join|"inner"か"outer"のどちらかを指定。デフォルトは"outer"。連結に用いない軸のインデックスとして、"inner"の場合は共通したインデックスのみを、"outer"の場合はインデックスの和集合を用いる|
|keys|オブジェクトに連結する際、連結対象の軸の方向に階層型インデックスを作成する場合に、各オブジェクトと紐づける値。任意の値のリストや配列や、タプルの配列、配列のリスト(複数階層の配列がlevelsに指定されている場合)を指定できる|
|levels|階層型インデックスの階層(複数のキーが指定されている場合は複数の階層)として使用するインデックスの指定|
|names|keysやlevelsを指定して階層型インデックスを作成した場合の、各階層の名前|
|verify_integrity|連結後のオブジェクトの新しい軸に重複があるかをチェックし、重複がある場合は例外を発生させる。デフォルトではFalseとなっており、重複を許す|
|ignore_index|連結する軸のインデックスを保存する、range(total_length)によって新たなインデックスを生成して設定する|

## 8.2.4  重複のあるデータの結合

データのマージ操作とも連結操作とも言い難いような状況もある  
2つのデータセットのインデックスの一部が重複しているか、完全に重複しているような場合になる

In [85]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])

In [86]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [87]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [88]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

NumPyのwhere関数は行列志向のif-else文で、if-elseの処理をベクトル化できる  
where関数に、if条件式、Trueのときの処理、Falseのときの処理の3つを渡すことで、要素ごとに審査してどちらかの処理を行った結果の配列を返す

In [89]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

シリーズにはcombine_firstというメソッドがあり、このメソッドを使うことで上のように欠損値をほかの配列から補う処理を行うが、where文と違い、インデックスをソートして整列されたデータを得られる

In [90]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})

In [91]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [92]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [93]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


データフレームに対して、combine_firstメソッドを使うと、列同士で同様の操作を行い、渡したデータフレームから欠損値の部分を補う動作をする  
渡されたオブジェクトから補完していると考えられる

# 8.3  変形とピボット操作

テーブル形式のデータを整形し直すための基本操作はいくつかあり、これらの操作を変形やピボット操作とも呼ばれる

## 8.3.1  階層型インデックスによる変形

階層型インデックスを用いると、データフレームに含まれるデータの形状を一貫した方法で変更できる  
基本となるアクションは以下の2つになる  
- stack  
データ内の各列を行へとピボット(回転)させる
- unstack  
各行を列へと回転させる

In [94]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"], name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


上のデータフレームを使って例を見ていく

In [95]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

データフレームに対してstackメソッドを使うと、列が行にピボットされ、上のように階層型インデックスのシリーズに変換される

In [96]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


先程の階層型インデックスを持つシリーズをデータフレームの形状にするには、データフレームにunstackメソッドを使うことでできる

In [97]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [98]:
result.unstack("state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


unstackメソッドはデフォルトだと一番内側の階層をunstackの対象とする  
番号やラベル名を渡して対象となる階層を指定することで、unstackの対象を変更することができる

In [99]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"])
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"])

In [100]:
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [101]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


unstackする際に、対象となる階層の値のうち一部が含まれていないサブグループがある場合は、含まれていない部分には欠損値が入れられる

In [102]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [103]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

stackメソッドで階層型インデックスのシリーズにするときには、デフォルトでは欠損値は除去されるが、キーワード引数のdropnaにFalseを渡すことで除去しないようにできる

In [104]:
df = pd.DataFrame({"left": result, "right": result +5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [105]:
df.unstack("state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

データフレームをunstackすると、unstackの対象となった階層が、生成されたデータフレームの列の最も低い階層に入る

In [106]:
df.unstack("state").stack("side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

階層型インデックスのデータフレームでstackを呼び出す際には、stackの対象となる軸をラベル名で指定することもできる

## 8.3.2  「縦持ち」フォーマットから「横持ち」フォーマットへのピボット

複数の時系列データをデータベースやCSVファイルに保存する方法としては、いわゆるlong(縦持ち)フォーマットや積み上げ型フォーマットがよく使われる

In [107]:
data = pd.read_csv(r".\download_file\examples\macrodata.csv")
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [108]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [109]:
columns = pd.Index(["realgdp", "infl", "unemp"], name="item")

In [110]:
data = data.reindex(columns=columns)
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [111]:
data.index = periods.to_timestamp("D", "end")
data.index

DatetimeIndex(['1959-03-31 23:59:59.999999999',
               '1959-06-30 23:59:59.999999999',
               '1959-09-30 23:59:59.999999999',
               '1959-12-31 23:59:59.999999999',
               '1960-03-31 23:59:59.999999999',
               '1960-06-30 23:59:59.999999999',
               '1960-09-30 23:59:59.999999999',
               '1960-12-31 23:59:59.999999999',
               '1961-03-31 23:59:59.999999999',
               '1961-06-30 23:59:59.999999999',
               ...
               '2007-06-30 23:59:59.999999999',
               '2007-09-30 23:59:59.999999999',
               '2007-12-31 23:59:59.999999999',
               '2008-03-31 23:59:59.999999999',
               '2008-06-30 23:59:59.999999999',
               '2008-09-30 23:59:59.999999999',
               '2008-12-31 23:59:59.999999999',
               '2009-03-31 23:59:59.999999999',
               '2009-06-30 23:59:59.999999999',
               '2009-09-30 23:59:59.999999999'],
              dtype=

In [112]:
ldata = data.stack().reset_index().rename(columns={0: "value"})
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


上では一連のデータの変形によって、もともとのdataからldataを得ることができた  
PeriodIndexはyearとquaterに渡した列を組み合わせて新しい型のデータ作成している  
dataを必要な列だけに絞り込み、行のインデックスに作成した日時のデータを指定  
その日時のインデックスを配列に変換して行インデックスを初期値に変更、列のラベル名に0が入ってしまっているので0からvalueに変更している

これが、2つ以上のキーを持つ複数の時系列データや他のデータをいわゆるlong(縦持ち)フォーマットで収めた例になる(この例だと、キーは日付(date)と項目(item))  
テーブルの各行が1回の観測を表す

MySQLのようなリレーショナルデータベースでは、上の例のように、固定のスキーマを持った(つまり列名とデータ型が固定された)形式でデータが格納される  
このような形式でデータベースに格納することで、テーブルにデータを追加していく際にitem列の値の種類に変化が生じても、データベースの構造を変える必要がなくなる  
上の例では、通常dateとitemが主キー(リレーショナルデータベース特有の言語)になる  
主キーによって関係の完全性が担保され、簡単に結合できるようになる  
しかし場合によっては、この縦持ちフォーマットでデータを取り扱うのが困難なことがある  
縦持ちフォーマットよりも、date列に含まれるタイムスタンプがインデックスとなっており、個々のitemの値(realgdp)ごとに1つの列が作られているデータベースの方が望ましい

In [113]:
pivoted = ldata.pivot("date", "item", "value")
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


データフレームのpivotメソッドに、行のインデックスとして使う列名、列のイデックスとして使いたい列名、値として使いたい列名、の3つを渡すことで、特定の列を行インデックスと列インデックスに変換したデータフレームを作成することができる

In [114]:
ldata["value2"] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.219342
1,1959-03-31 23:59:59.999999999,infl,0.000,0.282587
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.793531
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,1.455922
4,1959-06-30 23:59:59.999999999,infl,2.340,1.715724
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.242248
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.120912
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.166495
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.327635
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.267378


値を示す列が2つあるデータフレームを同時に変換したい場合を試してみる

In [115]:
pivoted = ldata.pivot("date", "item")
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.282587  0.219342   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.715724  1.455922   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.166495 -0.120912   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.903213  0.267378   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.277476  1.868079   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.793531  
1959-06-30 23:59:59.999999999 -0.242248  
1959-09-30 23:59:59.999999999 -0.327635  
1959-12-31 23:59:59.999999999 -1.976418  
1960-03-31 23:59:59.999999999  1.375126

In [116]:
pivoted["value"][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


pivotメソッドで最後の引数を省略すると、階層構造のある列を持ったデータフレームが得られる

In [117]:
unstacked = ldata.set_index(["date", "item"]).unstack("item")
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.282587  0.219342   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.715724  1.455922   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.166495 -0.120912   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.903213  0.267378   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.277476  1.868079   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.033516  1.335823   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.224876  2.016163   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.793531  
1959-06-30 23:59:59.999999999 -0.242248  
1959-09-30 23:59:59.999999999 -0.327635  
1959-12-31 23:59:59.999999999 -1.976418  
1960-03-31 23:59:59.999999999  1.375126  
1960-06-30 23:59:59.999999999 -1.199056  
1960-09-30 23:59:59.999999999 -0.695701

実は、set_indexメソッドで行インデックスにする列を指定して、unstackメソッドで片方の行インデックスを列インデックスに変換する動作は、pivotメソッドを使ったときと同じ動作をする

## 8.3.3  「横持ち」フォーマットから「縦持ち」フォーマットでのピボット

データフレームのpivotメソッドと逆を行うのがpandasのmelt関数になる  
pivotが1つの列を新たなデータフレームの複数の列へと分解するのに対し、pandasのmelt関数は複数の列を1つの列へとマージし、入力よりも長い縦持ちのデータフレームを生成する

In [118]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [119]:
melted = pd.melt(df, ["key"])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


pandasのmelt関数に、縦持ちフォーマットに変形したいデータフレームと、識別するための情報が入った列名(キーワード引数はid_vars)をリストとして渡すことでできる

In [120]:
reshaped = melted.pivot("key", "variable", "value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


pivotを用いて元の形状に戻すこともできる

In [121]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


上のpivotメソッドを使って取得したデータはkeyの列が行のインデックスとして変換している
インデックスのデータを行に戻して使いたい場合はreset_indexメソッドを使うことでできる

In [122]:
pd.melt(df, id_vars=["key"], value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


pandasのmelt関数でマージする際、デフォルトではキーとする列以外のすべての列が値の列として用いられるが、一部の列のみを持ちるように指定することもでき、キーワード引数のvalue_varsに使いたい列のリストを渡すことでできる  
この場合、指定されなかった列は無視される(上の場合C列)

In [123]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [124]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


pandasのmelt関数に識別情報を含む列(id_vars)を指定せずに使うこともできる